In [ ]:
import pandas as pd

# Load the CSV file
file_path = 'generated_data.csv'  # Change this to the path of your CSV file
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())

                             context                            question  \
0             The liver pumps water.             What liver pumps water?   
1       The diabetes controls sugar.       What diabetes controls sugar?   
2          The lungs controls sugar.          What lungs controls sugar?   
3           The liver pumps glucose.           What liver pumps glucose?   
4  The brain occurs when low oxygen.  What brain occurs when low oxygen?   

     answer  
0     liver  
1  diabetes  
2     lungs  
3     liver  
4     brain  


In [ ]:
from transformers import AutoTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Preprocessing function for tokenization
def preprocess_data(examples):
    # Tokenize context and question
    tokenized_input = tokenizer(examples['question'], examples['context'], truncation=True, padding="max_length", max_length=512)

    # Add start and end positions for question answering (use answer span)
    # We find the start and end positions of the answer in the context
    answer_start = examples['context'].find(examples['answer'])
    answer_end = answer_start + len(examples['answer'])

    tokenized_input['start_positions'] = answer_start
    tokenized_input['end_positions'] = answer_end

    return tokenized_input

# Apply tokenization to each row
df = df.apply(preprocess_data, axis=1)

In [ ]:
import torch
from torch.utils.data import Dataset

# Custom Dataset class for Question Answering
class QADataset(Dataset):
    def __init__(self, df):
        self.data = df
        self.input_ids = [item['input_ids'] for item in df]
        self.attention_mask = [item['attention_mask'] for item in df]
        self.start_positions = [item['start_positions'] for item in df]
        self.end_positions = [item['end_positions'] for item in df]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx]),
            'attention_mask': torch.tensor(self.attention_mask[idx]),
            'start_positions': torch.tensor(self.start_positions[idx]),
            'end_positions': torch.tensor(self.end_positions[idx])
        }

# Create Dataset
qa_dataset = QADataset(df)

# Check the dataset
print(qa_dataset[0])

{'input_ids': tensor([  101,  2054, 11290, 15856,  2300,  1029,   102,  1996, 11290, 15856,
         2300,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForQuestionAnswering

# Load the pre-trained BERT model for Question Answering
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,  # Number of epochs to train
    per_device_train_batch_size=80,  # Batch size per device
    per_device_eval_batch_size=80,  # Batch size for evaluation
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=qa_dataset,
    eval_dataset=qa_dataset,  # You can create a separate validation dataset if needed
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'qa_dataset' is not defined